    Python final

# A) DATA

In [33]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import holidays
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point

data = pd.read_parquet(Path("data") / "train.parquet") # import train 

meteo = pd.read_csv(Path("data") / "external_data.csv") # import meteo 

meteo.head()

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,hnuage1,nnuage2,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4
0,7149,2021-01-01 00:00:00,100810,80,1,270,1.8,272.75,272.15,96,...,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7149,2021-01-01 03:00:00,100920,110,3,300,1.7,271.25,270.95,98,...,1500.0,2.0,3.0,3000.0,NaN,NaN,NaN,NaN,NaN,NaN
2,7149,2021-01-01 06:00:00,100950,30,3,290,2.6,271.95,271.65,98,...,480.0,4.0,6.0,2000.0,6.0,3.0,3000.0,NaN,NaN,NaN
3,7149,2021-01-01 09:00:00,101100,150,2,280,1.7,272.45,272.05,97,...,1740.0,3.0,3.0,2800.0,NaN,NaN,NaN,NaN,NaN,NaN
4,7149,2021-01-01 12:00:00,101110,30,0,50,1.0,276.95,274.15,82,...,330.0,4.0,6.0,570.0,7.0,6.0,810.0,NaN,NaN,NaN


In [34]:
meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3322 entries, 0 to 3321
Data columns (total 59 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   numer_sta  3322 non-null   int64  
 1   date       3322 non-null   object 
 2   pmer       3322 non-null   int64  
 3   tend       3322 non-null   int64  
 4   cod_tend   3322 non-null   int64  
 5   dd         3322 non-null   int64  
 6   ff         3322 non-null   float64
 7   t          3322 non-null   float64
 8   td         3322 non-null   float64
 9   u          3322 non-null   int64  
 10  vv         3322 non-null   int64  
 11  ww         3322 non-null   int64  
 12  w1         3315 non-null   float64
 13  w2         3312 non-null   float64
 14  n          3166 non-null   float64
 15  nbas       3317 non-null   float64
 16  hbas       2869 non-null   float64
 17  cl         2909 non-null   float64
 18  cm         1941 non-null   float64
 19  ch         1678 non-null   float64
 20  pres    

# A-1) DONNEE METEO   
ETAPE 1: Filtrer les données (date et geographie et NaN)

1/ FILTRER PAR DATE

In [35]:
# DATE 

# Convertir les colonne en type datetime
meteo['date'] = pd.to_datetime(meteo['date'], format='%Y-%m-%d %H:%M:%S')
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')

# Trouver la date minimum et maximum de meteo et data
date_min_meteo = meteo['date'].min()
date_max_meteo = meteo['date'].max()
date_min_data = data['date'].min()
date_max_data = data['date'].max()

print("Meteo dataset:", date_max_meteo, date_min_meteo)
print("Data dataset:", date_max_data, date_min_data)

Meteo dataset: 2021-10-21 12:00:00 2020-09-01 00:00:00
Data dataset: 2021-09-09 23:00:00 2020-09-01 01:00:00


Donc, ces données overlap nos training, mais sont dans le meme format

In [36]:
# filtrer
meteo = meteo[(meteo['date'] >= date_min_data) & (meteo['date'] <= date_max_data)]

2/ FILTRER PAR GEOGRAPHIE 

On remarque qu'il n'y a aucune indication sur la geographie (region, departement, commune, longitude ou latitude)
DONC, on va HYPOTHESE IMPORTANTE: supposer que toutes les données concernent paris intra-muros (ou sont localisé nos bornes)

3/ GERER LES VALEURS MANQUANTES

In [37]:
# Gerer les valeurs manquantes
print("Lignes:", meteo.shape[0], "colonnes:", meteo.shape[1]) # trouver le nombre de lignes dans le dataset filtré

Lignes: 2988 colonnes: 59


In [38]:
meteo.isna().sum() # Savoir le nombre de Nan dans chaque colonnes

numer_sta       0
date            0
pmer            0
tend            0
cod_tend        0
dd              0
ff              0
t               0
td              0
u               0
vv              0
ww              0
w1              7
w2             10
n             138
nbas            5
hbas          410
cl            371
cm           1284
ch           1526
pres            0
niv_bar      2988
geop         2988
tend24         10
tn12         2241
tn24         2988
tx12         2241
tx24         2988
tminsol      2987
sw           2988
tw           2988
raf10          10
rafper          0
per             0
etat_sol       49
ht_neige       44
ssfrai        403
perssfrai     403
rr1             8
rr3             4
rr6            12
rr12           16
rr24           18
phenspe1     2988
phenspe2     2988
phenspe3     2988
phenspe4     2988
nnuage1       406
ctype1        720
hnuage1       412
nnuage2      1466
ctype2       1691
hnuage2      1466
nnuage3      2445
ctype3       2577
hnuage3   

In [39]:
# Ensuite, on retire toutes les colonnes qui sont remplis en totalité de valeurs manquantes
meteo = meteo.dropna(axis=1, how='all').copy()
print("Lignes:", meteo.shape[0], "colonnes:", meteo.shape[1]) # trouver le nombre de lignes dans le dataset filtré

# On a egalement remarque que dabs 'Tminsol' il n'y a qu'une valeur, on peut donc egalement supprimer cette colonne
meteo = meteo.drop('tminsol', axis=1).copy()
print("Lignes:", meteo.shape[0], "colonnes:", meteo.shape[1]) # check

Lignes: 2988 colonnes: 49
Lignes: 2988 colonnes: 48


4/ GERER LES COLONNES A MODALITE UNIQUE

In [40]:
meteo.nunique()

numer_sta       1
date         2987
pmer          461
tend           97
cod_tend        9
dd             37
ff            106
t             345
td            277
u              77
vv           1137
ww             42
w1              9
w2              9
n              10
nbas           10
hbas           91
cl             11
cm              9
ch             10
pres          458
tend24        340
tn12          215
tx12          276
raf10         168
rafper        182
per             1
etat_sol        9
ht_neige        7
ssfrai          3
perssfrai       2
rr1            31
rr3            58
rr6            84
rr12          118
rr24          152
nnuage1         9
ctype1          8
hnuage1       270
nnuage2         8
ctype2          8
hnuage2       235
nnuage3         8
ctype3          8
hnuage3       152
nnuage4         7
ctype4          5
hnuage4        24
dtype: int64

In [41]:
meteo = meteo.loc[:, meteo.nunique() > 1] # supprimer colonnes à modalité unique
meteo.info() # check

<class 'pandas.core.frame.DataFrame'>
Index: 2988 entries, 0 to 3321
Data columns (total 46 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       2988 non-null   datetime64[ns]
 1   pmer       2988 non-null   int64         
 2   tend       2988 non-null   int64         
 3   cod_tend   2988 non-null   int64         
 4   dd         2988 non-null   int64         
 5   ff         2988 non-null   float64       
 6   t          2988 non-null   float64       
 7   td         2988 non-null   float64       
 8   u          2988 non-null   int64         
 9   vv         2988 non-null   int64         
 10  ww         2988 non-null   int64         
 11  w1         2981 non-null   float64       
 12  w2         2978 non-null   float64       
 13  n          2850 non-null   float64       
 14  nbas       2983 non-null   float64       
 15  hbas       2578 non-null   float64       
 16  cl         2617 non-null   float64       
 17  

5/ GERER LES DUPLICATS

On remarque que le dataset a 2988 observations, hors il n'y a que 2987 modalités pour la date. Cela signifie qu'il y a un doublon. Verifions cela. 

In [42]:
duplicates = meteo[meteo['date'].duplicated(keep=False)]  # selectionner le doublons
print(duplicates) # Afficher les doublons

                    date    pmer  tend  cod_tend   dd   ff       t      td  \
2017 2020-11-20 18:00:00  103650    20         2  180  1.0  278.15  276.45   
2018 2020-11-20 18:00:00  103650    20         2  180  1.0  278.15  276.45   

       u     vv  ...  hnuage1  nnuage2  ctype2  hnuage2  nnuage3  ctype3  \
2017  89  25000  ...   7000.0      NaN     NaN      NaN      NaN     NaN   
2018  89  25000  ...   7000.0      NaN     NaN      NaN      NaN     NaN   

      hnuage3  nnuage4  ctype4  hnuage4  
2017      NaN      NaN     NaN      NaN  
2018      NaN      NaN     NaN      NaN  

[2 rows x 46 columns]


In [43]:
# Il y a donc bien un doublon concernant la date. Verifions si ce sont des doublons "parfaits"
identiques = duplicates.iloc[0].equals(duplicates.iloc[1]) # verifier l'egalité element par element
print("Perfectly identical observations? ", identiques) # resultats


Perfectly identical observations?  True


In [44]:
# On peut donc en supprimer un 
meteo = meteo.drop_duplicates(subset=['date'], keep='first') # garder que la 1ere occurence

ETAPE 2: Comprendre les données et les grouper en thèmes 

In [45]:
meteo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2987 entries, 0 to 3321
Data columns (total 46 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       2987 non-null   datetime64[ns]
 1   pmer       2987 non-null   int64         
 2   tend       2987 non-null   int64         
 3   cod_tend   2987 non-null   int64         
 4   dd         2987 non-null   int64         
 5   ff         2987 non-null   float64       
 6   t          2987 non-null   float64       
 7   td         2987 non-null   float64       
 8   u          2987 non-null   int64         
 9   vv         2987 non-null   int64         
 10  ww         2987 non-null   int64         
 11  w1         2980 non-null   float64       
 12  w2         2977 non-null   float64       
 13  n          2849 non-null   float64       
 14  nbas       2982 non-null   float64       
 15  hbas       2577 non-null   float64       
 16  cl         2616 non-null   float64       
 17  

On va creer des groupes de données par thèmes (etant donné que la variable "Date" est celle qui va nous permettre de faire le lien avec notre training dataset on la mets partout)
1) Pression
2) Nuage (et visibilité)
3) Precipitations
4) Vent
5) Temperature
6) Neige
7) Humidité
8) Etat du sol (1 donnée)
9) Autre (temps present, temps passé 1 et 2 )

In [46]:
# Creation des 9 sous-ensembles

# 1) Pression
pression_columns = ['date', 'pmer', 'tend', 'pres', 'tend24', 'cod_tend']  
pression  = meteo[pression_columns]

# 2) Nuage (et visibilité)
nuage_columns = ['date', 'vv', 'nbas', 'hbas', 'cl', 'cm', 'ch', 'n', 'nnuage1', 'ctype1',  'hnuage1', 'nnuage2', 'ctype2', 'hnuage2', 'nnuage3', 'ctype3', 'hnuage3', 'nnuage4', 'ctype4', 'hnuage4'] 
nuage = meteo[nuage_columns]

# 3) Précipitations
precip_columns = ['date', 'rr1', 'rr12', 'rr3', 'rr6', 'rr24']  
precip = meteo[precip_columns]

# 4) Vent
vent_columns = ['date', 'dd', 'ff', 'rafper', 'raf10']  
vent = meteo[vent_columns]

# 5) Température
temp_columns = ['date', 't', 'tn12', 'tx12']  
temp = meteo[temp_columns]

# 6) Neige
neige_columns = ['date', 'ssfrai', 'perssfrai', 'ht_neige']  
neige = meteo[neige_columns]

# 7) Humidité
humidite_columns = ['date', 'u', 'td'] 
humidite = meteo[humidite_columns]

# 8) Etat du sol (1 donnée)
etat_sol_columns = ['date', 'etat_sol'] 
sol = meteo[etat_sol_columns]

# 9) Autre (temps présent, temps passé 1 et 2)
autre_columns = ['date', 'ww', 'w1', 'w2']  # Autres informations liées au temps passé et présent
autre = meteo[autre_columns]

In [47]:
# Nombre de colonnes dans le dataset initial en excluant la colonne 'date'
initial_columns = len(meteo.columns) - 1  # Exclure 'date'

# Nombre total de colonnes dans les sous-ensembles
total_subsets_columns = (
    len(pression_columns) + 
    len(nuage_columns) + 
    len(precip_columns) + 
    len(vent_columns) + 
    len(temp_columns) + 
    len(neige_columns) + 
    len(humidite_columns) + 
    len(etat_sol_columns) + 
    len(autre_columns) - 9  # Exclure la colonne 'date' dans chaque sous-ensemble
)

# Vérification si la somme des colonnes dans les sous-ensembles est égale au nombre de colonnes initial
if initial_columns == total_subsets_columns:
    print("OK")
else:
    print("Problem")


OK


# A-2) DONNEE TRAIN   

In [48]:
#Cleaning

# 1. Handle Missing Data
# Check for missing values
missing_data = data.isnull().sum()
print("Missing data:\n", missing_data)

# Drop rows with missing values in crucial columns
data = data.dropna(subset=['bike_count', 'date', 'latitude', 'longitude'])

# 3. Remove Duplicates
print("Number of duplicates before removal:", data.duplicated().sum())
data = data.drop_duplicates(subset=['counter_id', 'date'])
print("Number of duplicates after removal:", data.duplicated().sum())

# 4. Handle Outliers (check negative values in bike_count)
if (data['bike_count'] < 0).any():
    print("Negative bike counts detected.")
    data['bike_count'] = data['bike_count'].clip(lower=0)  # Replace negative values with 0

# 5. Validate Geographic Coordinates
# Check for invalid latitude and longitude values
invalid_coords = data[(data['latitude'] < -90) | (data['latitude'] > 90) |
                      (data['longitude'] < -180) | (data['longitude'] > 180)]
if not invalid_coords.empty:
    print(f"Invalid coordinates detected:\n{invalid_coords}")
    data = data.dropna(subset=['latitude', 'longitude'])  # Optionally drop invalid coordinates

# 6. Clean Text Columns (remove extra spaces, convert to lowercase)
text_columns = ['counter_name', 'site_name', 'counter_technical_id']
for col in text_columns:
    data[col] = data[col].str.strip().str.lower()

Missing data:
 counter_id                   0
counter_name                 0
site_id                      0
site_name                    0
bike_count                   0
date                         0
counter_installation_date    0
coordinates                  0
counter_technical_id         0
latitude                     0
longitude                    0
log_bike_count               0
dtype: int64
Number of duplicates before removal: 0
Number of duplicates after removal: 0


In [49]:
# 7. modalités uniques
data.nunique()

counter_id                     56
counter_name                   56
site_id                        30
site_name                      30
bike_count                    998
date                         8974
counter_installation_date      22
coordinates                    30
counter_technical_id           30
latitude                       30
longitude                      30
log_bike_count                998
dtype: int64

In [50]:
# 1. Convert `date` column to datetime type
data['date'] = pd.to_datetime(data['date'])

# 2. Extract temporal components
data['date_only'] = data['date'].dt.date
data['hour_only'] = data['date'].dt.hour
data['day_of_week'] = data['date'].dt.dayofweek
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year

# 3. Add temporal indicators
data['is_weekend'] = data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
data['is_rush_hour'] = data['hour_only'].apply(lambda x: 1 if (7 <= x <= 9 or 17 <= x <= 19) else 0)
data['season'] = data['month'].apply(lambda x: 'winter' if x in [12, 1, 2] \
                                      else 'spring' if x in [3, 4, 5] \
                                      else 'summer' if x in [6, 7, 8] \
                                      else 'autumn')

# 4. Add holiday indicators
def vacation(date):
    summer_vacation = (date.month == 7 or date.month == 8)
    christmas_vacation = (date.month == 12 and date.day >= 20)
    return 1 if (summer_vacation or christmas_vacation) else 0

data['vacation'] = data['date'].apply(vacation)

# 5. Add public holidays using the holidays library
fr_holidays = holidays.France(years=data['year'].unique().tolist())
def is_public_holiday(date):
    return 1 if date in fr_holidays else 0

data['is_public_holiday'] = data['date'].apply(is_public_holiday)

data.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,...,date_only,hour_only,day_of_week,month,year,is_weekend,is_rush_hour,season,vacation,is_public_holiday
48321,100007049-102007049,28 boulevard diderot e-o,100007049,28 boulevard diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",y2h15027244,48.846028,...,2020-09-01,2,1,9,2020,0,0,autumn,0,0
48324,100007049-102007049,28 boulevard diderot e-o,100007049,28 boulevard diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",y2h15027244,48.846028,...,2020-09-01,3,1,9,2020,0,0,autumn,0,0
48327,100007049-102007049,28 boulevard diderot e-o,100007049,28 boulevard diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",y2h15027244,48.846028,...,2020-09-01,4,1,9,2020,0,0,autumn,0,0
48330,100007049-102007049,28 boulevard diderot e-o,100007049,28 boulevard diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",y2h15027244,48.846028,...,2020-09-01,15,1,9,2020,0,0,autumn,0,0
48333,100007049-102007049,28 boulevard diderot e-o,100007049,28 boulevard diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",y2h15027244,48.846028,...,2020-09-01,18,1,9,2020,0,1,autumn,0,0


In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 496827 entries, 48321 to 929187
Data columns (total 22 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   counter_id                 496827 non-null  category      
 1   counter_name               496827 non-null  object        
 2   site_id                    496827 non-null  int64         
 3   site_name                  496827 non-null  object        
 4   bike_count                 496827 non-null  float64       
 5   date                       496827 non-null  datetime64[us]
 6   counter_installation_date  496827 non-null  datetime64[us]
 7   coordinates                496827 non-null  category      
 8   counter_technical_id       496827 non-null  object        
 9   latitude                   496827 non-null  float64       
 10  longitude                  496827 non-null  float64       
 11  log_bike_count             496827 non-null  float64  

In [52]:
# Create features (X) and response variable (y)
X = data.drop(columns=['bike_count', 'log_bike_count', 'counter_id', 'site_id', 'coordinates', 'counter_technical_id', 'date_only']) 
y = data['log_bike_count']

In [53]:
# train-test split
data_train, data_test, target_train, target_test = train_test_split(
    X, y, random_state=2408)

# B) MODELES

Modèles testés:  
- Decision tree regressor
- Random forest regressor

In [67]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from dirty_cat import TableVectorizer
from sklearn.pipeline import make_pipeline


STEP 1: Creer toutes les combinaisons possibles de features


# B-1) Decision tree regressor

In [76]:
# Select only numerical features
data_train_num = data_train.select_dtypes(include="number")
data_train_cat = data_train.select_dtypes(include="object")

In [69]:
# 1 - FIND BEST PARAMETERS

# store parameters to test in a dictionnary
param_grid = {'max_depth': range(10, 15), 
    'min_samples_split': range(1,3),  
    'min_samples_leaf': range(8,16)} 

# initialize the decision tree regressor
dtr_model = DecisionTreeRegressor(random_state=2408)

# initialize GridSearchCV
grid_search = GridSearchCV(dtr_model, param_grid=param_grid, n_jobs=4, cv=5, return_train_score=True)

# fit GridSearchCV
grid_search.fit(data_train_num, target_train) # REGLER CE PB, TU PRENDS QUE LES NUMERIQUES

# get the best max_depth
best_max_depth = grid_search.best_params_['max_depth']
best_split = grid_search.best_params_['min_samples_split']
best_leaf = grid_search.best_params_['min_samples_leaf']

print(best_max_depth, best_split, best_leaf)

/Users/Audrey/bike_counters-AM-/env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
200 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
200 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/Audrey/bike_counters-AM-/env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/Audrey/bike_counters-AM-/env/lib/python3.8/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/Users/Audrey/bike_counters-AM-/env/lib/python3.8/site-packages/sklearn/base.py", line 638, in _validate_params


14 2 13


In [73]:
# DEFINE PIPELINES and test RMSE - WITHOUT SCALER

# Pipeline
pipeline_no_scaler_dtr = make_pipeline(TableVectorizer(),
    DecisionTreeRegressor(max_depth=best_max_depth,min_samples_split=best_split,min_samples_leaf=best_leaf)
)

# RMSE Without scaler
pipeline_no_scaler_dtr.fit(data_train, target_train)
target_pred_no_scaler = pipeline_no_scaler_dtr.predict(data_test)
rmse_no_scaler = mean_squared_error(target_test, target_pred_no_scaler, squared=False)
print(f"RMSE without scaler: {rmse_no_scaler}")

RMSE without scaler: 0.608236986170773


In [86]:
# DEFINE PIPELINES and test RMSE - WITH SCALER

# split numerical and categorical
dfcat= X.select_dtypes(include="object").columns
dfnum= X.select_dtypes(include="number").columns


# define ColumnTransformer
preprocessor = ColumnTransformer([
    ('table_vectorizer', TableVectorizer(),
     dfcat),
    ('standard-scaler', StandardScaler(), dfnum)
])

# with scaler
pipeline_scaler_dtr = make_pipeline(
    preprocessor,
    DecisionTreeRegressor(max_depth=best_max_depth, min_samples_split=best_split, min_samples_leaf=best_leaf)
)

# RMSE with scaler
pipeline_scaler_dtr.fit(data_train, target_train)
target_pred_scaler = pipeline_scaler_dtr.predict(data_test)
rmse_scaler = mean_squared_error(target_test, target_pred_scaler, squared=False)
print(f"RMSE with scaler: {rmse_scaler}")

RMSE with scaler: 0.6058565069881058


STEP 2: Encoding: Table Vectorizer 